In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [1]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik

# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

LENGTH = 2648
RECS = 1384 * 9
TRANSL = 307

X = np.zeros([RECS, LENGTH, 20], np.float32)
Y = np.zeros([RECS, TRANSL], np.int16)
counter = 0

for i in range(9):
    Xpart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-trans-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()


/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


12429 12456


In [ ]:
mc = keras.callbacks.ModelCheckpoint('models/mfcc-ctc-{epoch:08d}.h5', 
                                     save_weights_only=False, period=5)

def mk_model(max_label_length):
    feature_input = Input(shape = (None, NFEATS))
    layer = Lambda(lambda x: (x - MEAN) / STD)(feature_input)
    layer = Lambda(K.expand_dims)(layer)
    layer_1 = Conv2D(12, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer_1 = Conv2D(16, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer_1 = Conv2D(24, [7,1], activation='linear', kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer = TimeDistributed(Flatten())(layer)
    layer_10 = LSTM(256, return_sequences = True, kernel_initializer=Orthogonal())(layer)
    layer_11 = LSTM(192, return_sequences = True, kernel_initializer=Orthogonal())(layer_10)
    layer_12 = LSTM(160, return_sequences = True, kernel_initializer=Orthogonal())(layer_11)
    layer_13 = LSTM(128, return_sequences = True, kernel_initializer=Orthogonal())(layer_12)
    layer_15 = LSTM(NPHONES + 1, return_sequences = True, activation = 'softmax')(layer_13)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, layer_15, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, layer_15)
    return model, predictive

def train(model, trainX, trainy, trainXl, trainyl, epochs = 50):
    # important: batch_size=1 bugs Tensorflow
    optimizer = Adam(0.0003, clipnorm=1.)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    return model.fit([trainX, trainy, trainXl, trainyl], np.zeros(trainX.shape[0]), epochs = epochs,
                     batch_size = 32, callbacks=[mc])

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    NPHONES = Y.max()
    NFEATS = data[0].shape[2]
    X_lens = np.array([try_else(
            (lambda:np.where((x).mean(1) == (x).std(1))[0][0]),
            X.shape[1])
        for x in X])
    X_lens = np.ceil(X_lens / 4.0)
    Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
    Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens])
    data = data[0][np.where(Y_lens)], data[1][np.where(Y_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)
    print(list(map(lambda x:x.shape, data)))
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=1000) # at 300 it makes sensible predictions
    valid_data = make_data(*load_data(VALIDDATA))
    predictions = predict.predict(valid_data[0])
    DERs = validate(predictions, valid_data[2], valid_data[1], valid_data[3])
    print("Validation Digit Error Rate: {}".format(sum([x[0] * x[1] for x in DERs]) / sum([x[1] for x in DERs])))
    predict.save('cyfry.pred.h5')

[(12416, 2648, 20), (12416, 307), (12416, 1), (12416, 1)]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 20)     0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, None, 20)     0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, None, 20, 1)  0           lambda_3[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, None, 20, 12) 72          lambda_4[0][0]                   
___________________________________________________

In [13]:
GRU(np.zeros(NPHONES + 1))(Input((30,10)))

TypeError: only size-1 arrays can be converted to Python scalars

In [4]:
pr = predict.predict(X[:1])
pr = pr.argmax(2)[0]
pr

array([20, 20, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 11, 37,  6, 37, 37, 37, 28, 28, 28, 37,  4, 37, 29, 37, 37,
       28, 37, 37, 27, 37, 31, 37, 21, 37, 37, 37, 34, 37,  0, 37,  6, 37,
        4, 37, 37, 33, 37, 14, 28, 37, 33, 37,  7, 37, 33, 37, 14, 37, 29,
       37, 37, 23, 37,  8, 37, 30, 37, 37, 37, 37,  4, 37, 37,  0, 37,  6,
       37, 37,  4, 37, 37, 33, 37,  0, 37, 37, 37, 37, 30, 37, 37,  6, 37,
       37, 37, 20, 20, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 31,
       31, 37, 37, 28, 28, 37, 37, 37, 37, 29, 37, 28, 28, 37, 37,  6, 37,
       10, 37, 37, 37, 37, 37, 37, 37,  3, 37, 37, 29, 37, 37, 37, 37, 28,
       37, 37, 37, 37, 12, 37, 37,  6, 37, 37,  0, 37, 37, 22, 37,  6, 37,
       28, 28,  4,  4, 37

In [5]:
pr = predict.predict(X[:1])
pr = K.ctc_decode(pr, [271])[0][0]
pr

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


<tf.Tensor 'SparseToDense:0' shape=(1, 56) dtype=int64>

In [6]:
pred_tr = pr.eval(session=K.get_session())

In [7]:
Y[:1]

array([[20,  3, 31, 28,  4, 29, 20, 28, 34, 31, 21,  8, 34,  0,  6, 20,
        34,  8, 28, 15, 24, 33, 14, 29, 23,  8, 10, 20,  8, 34, 20,  0,
         6, 34, 33, 20,  0, 30,  6, 20, 31, 20, 28, 28, 29, 28,  6, 20,
        10, 27,  3, 29, 30,  0, 20, 12, 29,  5, 22,  6, 20, 28,  4, 21,
         6,  4, 20, 31, 36, 20, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [8]:
import editdistance
editdistance.eval(list(pred_tr[0]), list(Y[0, :70]))

28

In [9]:
pred_tr[0]

array([20, 11,  6, 28,  4, 29, 28, 27, 31, 21, 34,  0,  6,  4, 33, 14, 28,
       33,  7, 33, 14, 29, 23,  8, 30,  4,  0,  6,  4, 33,  0, 30,  6, 20,
       31, 28, 29, 28,  6, 10,  3, 29, 28, 12,  6,  0, 22,  6, 28,  4, 21,
        6,  4, 31, 36, 20])

In [10]:
preds = predict.predict(X[:160])

In [11]:
pr = K.ctc_decode(preds, X_lens[:160])[0][0]

In [12]:
pr.eval(session=K.get_session())

array([[20, 11,  6, ..., -1, -1, -1],
       [20, 24, 22, ..., -1, -1, -1],
       [20, 10, 29, ..., -1, -1, -1],
       ...,
       [20, 12, 33, ..., -1, -1, -1],
       [20, 22,  6, ..., -1, -1, -1],
       [20, 28, 31, ..., -1, -1, -1]])

In [13]:
hypos = [list(x) for x in pr.eval(session=K.get_session())]
lens = [x.index(-1) if -1 in x else len(x) for x in hypos]
hypos = [x[:lim] for x, lim in zip(hypos, lens)]

In [14]:
gts = [list(x) for x in Y[:160]]
lens = [x.index(37) if 37 in x else len(x) for x in gts]
gts = [x[:lim] for x, lim in zip(gts, lens)]

In [15]:
[editdistance.eval(gt, hypo) / len(gt) for gt, hypo in zip(gts, hypos)]

[0.4,
 0.32558139534883723,
 0.3225806451612903,
 0.2875,
 0.34710743801652894,
 0.23404255319148937,
 0.2962962962962963,
 0.25555555555555554,
 0.23529411764705882,
 0.2956521739130435,
 0.3291139240506329,
 0.2361111111111111,
 0.23076923076923078,
 0.4117647058823529,
 0.34615384615384615,
 0.5,
 0.32894736842105265,
 0.35365853658536583,
 0.30337078651685395,
 0.3855421686746988,
 0.23214285714285715,
 0.2558139534883721,
 0.2268041237113402,
 0.2549019607843137,
 0.25,
 0.32673267326732675,
 0.2727272727272727,
 0.3333333333333333,
 0.2361111111111111,
 0.30303030303030304,
 0.25842696629213485,
 0.3333333333333333,
 0.32098765432098764,
 0.3283582089552239,
 0.25,
 0.34615384615384615,
 0.26506024096385544,
 0.3291139240506329,
 0.3488372093023256,
 0.2413793103448276,
 0.20987654320987653,
 0.17857142857142858,
 0.20481927710843373,
 0.2413793103448276,
 0.3956043956043956,
 0.30434782608695654,
 0.23943661971830985,
 0.2911392405063291,
 0.3918918918918919,
 0.4210526315789473

In [16]:
hypos

[[20,
  11,
  6,
  28,
  4,
  29,
  28,
  27,
  31,
  21,
  34,
  0,
  6,
  4,
  33,
  14,
  28,
  33,
  7,
  33,
  14,
  29,
  23,
  8,
  30,
  4,
  0,
  6,
  4,
  33,
  0,
  30,
  6,
  20,
  31,
  28,
  29,
  28,
  6,
  10,
  3,
  29,
  28,
  12,
  6,
  0,
  22,
  6,
  28,
  4,
  21,
  6,
  4,
  31,
  36,
  20],
 [20,
  24,
  22,
  31,
  36,
  8,
  14,
  34,
  29,
  32,
  28,
  15,
  34,
  34,
  0,
  33,
  18,
  33,
  21,
  33,
  9,
  6,
  10,
  18,
  6,
  1,
  29,
  32,
  6,
  14,
  29,
  32,
  30,
  33,
  14,
  29,
  30,
  28,
  4,
  33,
  4,
  15,
  12,
  22,
  8,
  6,
  22,
  20,
  30,
  31,
  10,
  22,
  31,
  32,
  33,
  10,
  17,
  31,
  28,
  9,
  8,
  14,
  10,
  29,
  11,
  26,
  8,
  24,
  29,
  0,
  3,
  8,
  20],
 [20,
  10,
  29,
  0,
  21,
  6,
  17,
  31,
  24,
  8,
  1,
  9,
  15,
  20,
  8,
  17,
  1,
  29,
  7,
  33,
  1,
  15,
  19,
  8,
  17,
  9,
  29,
  36,
  33,
  10,
  6,
  1,
  31,
  26,
  27,
  28,
  4,
  29,
  4,
  31,
  14,
  30,
  31,
  32,
  3,
  31,
  

In [17]:
trn.evaluate(list(data), [np.zeros(len(data[0]))])

12416/12416 [==============================] - 292s 24ms/step


124.36646541123538

In [18]:
trn

In [22]:
trn.save("models/first_mfcc_124loss.h5")